In [98]:
import pandas as pd
import numpy as np
import binarymap
import dms_variants.globalepistasis

def normalize(df):

    mu = df.func_score.mean()
    std = df.func_score.std()

    df["func_score_norm"] = (df["func_score"] - mu) / std
    df["func_score_var_norm"] = (df["func_score_var"]) / std / std

    mu = df[df.n_aa_substitutions == 0].func_score.mean()

    df["func_score_centered"] = (df["func_score"] - mu) / std

In [99]:
exp = pd.read_csv("data/raw/covid-expression.csv")

In [7]:
# https://github.com/jbloomlab/SARS-CoV-2-RBD_DMS/blob/master/results/summary/global_epistasis_expression.md
exp.rename(
    columns={"delta_ML_meanF": "func_score", "var_ML_meanF": "func_score_var"},
    inplace=True,
)
exp = exp[pd.notnull(exp["func_score"])]
exp.fillna("", inplace=True)
exp["aa_substitutions"] = exp["aa_substitutions"].str.replace(" ", ":")

# fix 0 variance
exp.func_score_var.replace(0, 0.1, inplace=True)
normalize(exp)

In [8]:
pheno = "func_score_centered"
df = exp
df = df.assign(
    aa_substitutions=(
        df["aa_substitutions"]
        .replace(np.nan, "")
        .str
        .replace(":", " ")
    ),
    func_score=df[pheno],
)

In [10]:
alphabet = [
        "A",
        "C",
        "D",
        "E",
        "F",
        "G",
        "H",
        "I",
        "K",
        "L",
        "M",
        "N",
        "P",
        "Q",
        "R",
        "S",
        "T",
        "V",
        "W",
        "Y",
        "X",
        "*",
    ]

In [94]:
bmap = binarymap.BinaryMap(
    df,
    func_score_var_col="func_score_var"
)


df["aa_substitutions"] = df["aa_substitutions"].apply(
            lambda x: " ".join([xx for xx in x.split(" ") if xx in bmap.all_subs])
        )

In [20]:
ifit_df = dms_variants.globalepistasis.fit_models(bmap, "Gaussian")
ge = ifit_df.loc[0].model

In [90]:
bmap.sub_str_to_indices("K94N")

[1865]

In [93]:
bmap.binary_variants.shape

(177759, 4002)

In [65]:
bmap.

[254]

In [22]:
pred = ge.add_phenotypes_to_df(df, unknown_as_nan=False)

In [23]:
pred

,library,target,barcode,variant_call_support,total_count,ML_meanF,func_score,func_score_var,variant_class,aa_substitutions,n_aa_substitutions,func_score_norm,func_score_var_norm,func_score_centered,latent_phenotype,observed_phenotype
1,lib1,SARS-CoV-2,AAAAAAAAACTTAAAT,2,64.71,7.45,-2.134644,0.04,>1 nonsynonymous,N13S L60P K94N S147T C150Y,5,-0.588367,0.020117,-2.134644,-0.434895,-2.017980
2,lib1,SARS-CoV-2,AAAAAAAAACTTCAAT,5,117.96,7.92,-1.801331,0.03,>1 nonsynonymous,A22C R127G E141D L188V,4,-0.255053,0.015088,-1.801331,-0.171387,-1.538173
3,lib1,SARS-CoV-2,AAAAAAAACAAGCAGA,6,244.34,8.93,-1.085061,0.01,1 nonsynonymous,N13F,1,0.461217,0.005029,-1.085061,-0.146870,-1.378264
4,lib1,SARS-CoV-2,AAAAAAAACAATATAA,1,95.35,6.21,-3.014025,0.03,>1 nonsynonymous,C6K T15W K94Y V103W,4,-1.467747,0.015088,-3.014025,-5.212113,-2.319955
5,lib1,SARS-CoV-2,AAAAAAAACAGGTTGC,4,212.43,7.73,-1.936074,0.02,>1 nonsynonymous,V71K P149L N157T,3,-0.389797,0.010059,-1.936074,-0.325102,-2.011040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195076,lib2,SARS-CoV-2,TTTTTTTACAGTCTTG,12,295.44,8.49,-1.340365,0.02,1 nonsynonymous,Y165S,1,0.205913,0.010059,-1.340365,-0.142369,-1.346433
195077,lib2,SARS-CoV-2,TTTTTTTACCAGACTG,14,423.57,7.76,-1.858065,0.01,>1 nonsynonymous,G83V D98L,2,-0.311787,0.005029,-1.858065,-0.205175,-1.720954
195078,lib2,SARS-CoV-2,TTTTTTTCTTTAAAAA,7,159.59,7.68,-1.914799,0.02,>1 nonsynonymous,V71H S147Q,2,-0.368522,0.010059,-1.914799,-0.715666,-2.035728
195079,lib2,SARS-CoV-2,TTTTTTTGAAGCCCCA,8,98.22,10.38,-0.000018,0.03,wildtype,,0,1.546259,0.015088,-0.000018,0.000000,0.049386


In [21]:
df = pd.read_csv(input[0])
df = df.assign(
    aa_substitutions=(
        df[cget("substitutions", "substitutions")]
        .replace(np.nan, "")
        .str
        .replace(":", " ")
    ),
    func_score=df[phenotype],
)




alphabet = cget("alphabet", [
        "A",
        "C",
        "D",
        "E",
        "F",
        "G",
        "H",
        "I",
        "K",
        "L",
        "M",
        "N",
        "P",
        "Q",
        "R",
        "S",
        "T",
        "V",
        "W",
        "Y",
        "X",
        "*",
    ]
)

bmap = dms_variants.binarymap.BinaryMap(
    df[df.cv != int(wildcards.cv)],
    alphabet=alphabet,
    func_score_var_col="func_score_var" if noise is not None else None,
)

ifit_df = dms_variants.globalepistasis.fit_models(bmap, "Gaussian")
ge = ifit_df.loc[0].model

val = df[df.cv == int(wildcards.cv)].copy()

# fix for missing subs
val["aa_substitutions"] = val["aa_substitutions"].apply(
    lambda x: " ".join([xx for xx in x.split(" ") if xx in ge.binarymap.all_subs])
)

pred = ge.add_phenotypes_to_df(val, unknown_as_nan=False)
pred.dropna(subset=["observed_phenotype"], inplace=True)
pred.to_csv(
    output[1],
    index=False,
)

,library,target,barcode,variant_call_support,total_count,ML_meanF,func_score,func_score_var,variant_class,aa_substitutions,n_aa_substitutions,func_score_norm,func_score_var_norm,func_score_centered
1,lib1,SARS-CoV-2,AAAAAAAAACTTAAAT,2,64.71,7.45,-3.01,0.04,>1 nonsynonymous,N13S:L60P:K94N:S147T:C150Y,5,-0.588367,0.020117,-2.134644
2,lib1,SARS-CoV-2,AAAAAAAAACTTCAAT,5,117.96,7.92,-2.54,0.03,>1 nonsynonymous,A22C:R127G:E141D:L188V,4,-0.255053,0.015088,-1.801331
3,lib1,SARS-CoV-2,AAAAAAAACAAGCAGA,6,244.34,8.93,-1.53,0.01,1 nonsynonymous,N13F,1,0.461217,0.005029,-1.085061
4,lib1,SARS-CoV-2,AAAAAAAACAATATAA,1,95.35,6.21,-4.25,0.03,>1 nonsynonymous,C6K:T15W:K94Y:V103W,4,-1.467747,0.015088,-3.014025
5,lib1,SARS-CoV-2,AAAAAAAACAGGTTGC,4,212.43,7.73,-2.73,0.02,>1 nonsynonymous,V71K:P149L:N157T,3,-0.389797,0.010059,-1.936074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195076,lib2,SARS-CoV-2,TTTTTTTACAGTCTTG,12,295.44,8.49,-1.89,0.02,1 nonsynonymous,Y165S,1,0.205913,0.010059,-1.340365
195077,lib2,SARS-CoV-2,TTTTTTTACCAGACTG,14,423.57,7.76,-2.62,0.01,>1 nonsynonymous,G83V:D98L,2,-0.311787,0.005029,-1.858065
195078,lib2,SARS-CoV-2,TTTTTTTCTTTAAAAA,7,159.59,7.68,-2.70,0.02,>1 nonsynonymous,V71H:S147Q,2,-0.368522,0.010059,-1.914799
195079,lib2,SARS-CoV-2,TTTTTTTGAAGCCCCA,8,98.22,10.38,0.00,0.03,wildtype,,0,1.546259,0.015088,-0.000018
